https://auto.gluon.ai/stable/tutorials/tabular/advanced/tabular-deployment.html

In [1]:
# Need to do this for each autogluon notebook...
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could ta

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
label = 'class'
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv | Columns = 15 / 15 | Rows = 39073 -> 39073


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
6118,51,Private,39264,Some-college,10,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,>50K
23204,58,Private,51662,10th,6,Married-civ-spouse,Other-service,Wife,White,Female,0,0,8,United-States,<=50K
29590,40,Private,326310,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,44,United-States,<=50K
18116,37,Private,222450,HS-grad,9,Never-married,Sales,Not-in-family,White,Male,0,2339,40,El-Salvador,<=50K
33964,62,Private,109190,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,15024,0,40,United-States,>50K


In [3]:
save_path = 'agModels-predictClass-deployment'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Jun 27 20:43:36 UTC 2024
CPU Count:          4
Memory Avail:       30.05 GB / 31.36 GB (95.8%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will 

In [4]:
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]  # values to predict
test_data.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States,<=50K
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States,<=50K
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States,>50K
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States,<=50K
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States,<=50K


In [5]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data)
y_pred

0        <=50K
1        <=50K
2         >50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object

In [6]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestGini,0.842870,0.84,accuracy,0.199703,0.098500,1.336537,0.199703,0.098500,1.336537,1,True,5
1,CatBoost,0.842461,0.85,accuracy,0.022597,0.010295,2.246335,0.022597,0.010295,2.246335,1,True,7
2,RandomForestEntr,0.841130,0.83,accuracy,0.198707,0.107198,1.215840,0.198707,0.107198,1.215840,1,True,6
3,XGBoost,0.840925,0.86,accuracy,0.081970,0.010971,0.382895,0.081970,0.010971,0.382895,1,True,11
4,WeightedEnsemble_L2,0.840925,0.86,accuracy,0.084975,0.012574,0.578995,0.003005,0.001603,0.196100,2,True,14
5,LightGBM,0.839799,0.85,accuracy,0.048116,0.007437,0.475711,0.048116,0.007437,0.475711,1,True,4
6,LightGBMXT,0.836421,0.83,accuracy,0.024955,0.008415,1.843791,0.024955,0.008415,1.843791,1,True,3
7,ExtraTreesEntr,0.833862,0.81,accuracy,0.224840,0.094644,1.136293,0.224840,0.094644,1.136293,1,True,9
8,ExtraTreesGini,0.833862,0.82,accuracy,0.235396,0.086608,1.176014,0.235396,0.086608,1.176014,1,True,8
9,NeuralNetTorch,0.833555,0.83,accuracy,0.089375,0.017920,4.074289,0.089375,0.017920,4.074289,1,True,12


In [7]:
save_path_clone = save_path + '-clone'
# will return the path to the cloned predictor, identical to save_path_clone
path_clone = predictor.clone(path=save_path_clone)

Cloned TabularPredictor located in 'agModels-predictClass-deployment' to 'agModels-predictClass-deployment-clone'.
	To load the cloned predictor: predictor_clone = TabularPredictor.load(path="agModels-predictClass-deployment-clone")


In [8]:
predictor_clone = TabularPredictor.load(path=path_clone)
# You can alternatively load the cloned TabularPredictor at the time of cloning:
# predictor_clone = predictor.clone(path=save_path_clone, return_clone=True)

In [9]:
y_pred_clone = predictor.predict(test_data)
y_pred_clone

0        <=50K
1        <=50K
2         >50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object

In [10]:
y_pred.equals(y_pred_clone)

True

In [11]:
predictor_clone.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestGini,0.842870,0.84,accuracy,0.183619,0.098500,1.336537,0.183619,0.098500,1.336537,1,True,5
1,CatBoost,0.842461,0.85,accuracy,0.014817,0.010295,2.246335,0.014817,0.010295,2.246335,1,True,7
2,RandomForestEntr,0.841130,0.83,accuracy,0.165920,0.107198,1.215840,0.165920,0.107198,1.215840,1,True,6
3,XGBoost,0.840925,0.86,accuracy,0.077698,0.010971,0.382895,0.077698,0.010971,0.382895,1,True,11
4,WeightedEnsemble_L2,0.840925,0.86,accuracy,0.080998,0.012574,0.578995,0.003300,0.001603,0.196100,2,True,14
5,LightGBM,0.839799,0.85,accuracy,0.038970,0.007437,0.475711,0.038970,0.007437,0.475711,1,True,4
6,LightGBMXT,0.836421,0.83,accuracy,0.022115,0.008415,1.843791,0.022115,0.008415,1.843791,1,True,3
7,ExtraTreesGini,0.833862,0.82,accuracy,0.165002,0.086608,1.176014,0.165002,0.086608,1.176014,1,True,8
8,ExtraTreesEntr,0.833862,0.81,accuracy,0.168831,0.094644,1.136293,0.168831,0.094644,1.136293,1,True,9
9,NeuralNetTorch,0.833555,0.83,accuracy,0.092238,0.017920,4.074289,0.092238,0.017920,4.074289,1,True,12


In [12]:
predictor_clone.refit_full()

predictor_clone.leaderboard(test_data)

Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix "_FULL" and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `predictor.fit` call.
	To learn more, refer to the `.refit_full` method docstring which explains how "_FULL" models differ from normal models.
Fitting 1 L1 models ...
Fitting model: KNeighborsUnif_FULL ...
	0.01s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: KNeighborsDist_FULL ...
	0.01s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: LightGBMXT_FULL ...
	0.39s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: LightGBM_FULL ...
	0.29s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: RandomForestGini_FULL ...
	1.18s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: RandomForestEntr_FULL ...
	1.19s	 = Training   runtime
Fitting 1 L1 models ...
Fitting mode

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_FULL,0.842870,NaN,accuracy,0.013500,NaN,0.043275,0.013500,NaN,0.043275,1,True,21
1,RandomForestGini,0.842870,0.84,accuracy,0.174878,0.098500,1.336537,0.174878,0.098500,1.336537,1,True,5
2,CatBoost,0.842461,0.85,accuracy,0.013732,0.010295,2.246335,0.013732,0.010295,2.246335,1,True,7
3,RandomForestEntr,0.841130,0.83,accuracy,0.165156,0.107198,1.215840,0.165156,0.107198,1.215840,1,True,6
4,XGBoost,0.840925,0.86,accuracy,0.082682,0.010971,0.382895,0.082682,0.010971,0.382895,1,True,11
5,WeightedEnsemble_L2,0.840925,0.86,accuracy,0.085759,0.012574,0.578995,0.003077,0.001603,0.196100,2,True,14
6,LightGBM_FULL,0.840823,NaN,accuracy,0.045902,NaN,0.292713,0.045902,NaN,0.292713,1,True,18
7,LightGBM,0.839799,0.85,accuracy,0.039228,0.007437,0.475711,0.039228,0.007437,0.475711,1,True,4
8,RandomForestGini_FULL,0.839390,NaN,accuracy,0.162801,NaN,1.180342,0.162801,NaN,1.180342,1,True,19
9,RandomForestEntr_FULL,0.839185,NaN,accuracy,0.164247,NaN,1.190927,0.164247,NaN,1.190927,1,True,20


In [13]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestGini,0.842870,0.84,accuracy,0.163229,0.098500,1.336537,0.163229,0.098500,1.336537,1,True,5
1,CatBoost,0.842461,0.85,accuracy,0.012805,0.010295,2.246335,0.012805,0.010295,2.246335,1,True,7
2,RandomForestEntr,0.841130,0.83,accuracy,0.162952,0.107198,1.215840,0.162952,0.107198,1.215840,1,True,6
3,XGBoost,0.840925,0.86,accuracy,0.077232,0.010971,0.382895,0.077232,0.010971,0.382895,1,True,11
4,WeightedEnsemble_L2,0.840925,0.86,accuracy,0.080092,0.012574,0.578995,0.002860,0.001603,0.196100,2,True,14
5,LightGBM,0.839799,0.85,accuracy,0.038647,0.007437,0.475711,0.038647,0.007437,0.475711,1,True,4
6,LightGBMXT,0.836421,0.83,accuracy,0.020080,0.008415,1.843791,0.020080,0.008415,1.843791,1,True,3
7,ExtraTreesEntr,0.833862,0.81,accuracy,0.153894,0.094644,1.136293,0.153894,0.094644,1.136293,1,True,9
8,ExtraTreesGini,0.833862,0.82,accuracy,0.163230,0.086608,1.176014,0.163230,0.086608,1.176014,1,True,8
9,NeuralNetTorch,0.833555,0.83,accuracy,0.083057,0.017920,4.074289,0.083057,0.017920,4.074289,1,True,12


In [14]:
save_path_clone_opt = save_path + '-clone-opt'
# will return the path to the cloned predictor, identical to save_path_clone_opt
path_clone_opt = predictor.clone_for_deployment(path=save_path_clone_opt)

Cloned TabularPredictor located in 'agModels-predictClass-deployment' to 'agModels-predictClass-deployment-clone-opt'.
	To load the cloned predictor: predictor_clone = TabularPredictor.load(path="agModels-predictClass-deployment-clone-opt")
Clone: Keeping minimum set of models required to predict with best model 'WeightedEnsemble_L2'...
Deleting model KNeighborsUnif. All files under agModels-predictClass-deployment-clone-opt/models/KNeighborsUnif will be removed.
Deleting model KNeighborsDist. All files under agModels-predictClass-deployment-clone-opt/models/KNeighborsDist will be removed.
Deleting model LightGBMXT. All files under agModels-predictClass-deployment-clone-opt/models/LightGBMXT will be removed.
Deleting model LightGBM. All files under agModels-predictClass-deployment-clone-opt/models/LightGBM will be removed.
Deleting model RandomForestGini. All files under agModels-predictClass-deployment-clone-opt/models/RandomForestGini will be removed.
Deleting model RandomForestEntr.

In [15]:
predictor_clone_opt = TabularPredictor.load(path=path_clone_opt)

In [16]:
predictor_clone_opt.persist()

Persisting 2 models in memory. Models will require 0.0% of memory.


['XGBoost', 'WeightedEnsemble_L2']

In [17]:
y_pred_clone_opt = predictor_clone_opt.predict(test_data)
y_pred_clone_opt

0        <=50K
1        <=50K
2         >50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object

In [18]:
predictor_clone_opt.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost,0.840925,0.86,accuracy,0.068242,0.010971,0.382895,0.068242,0.010971,0.382895,1,True,1
1,WeightedEnsemble_L2,0.840925,0.86,accuracy,0.069860,0.012574,0.578995,0.001618,0.001603,0.196100,2,True,2


In [19]:
size_original = predictor.disk_usage()
size_opt = predictor_clone_opt.disk_usage()
print(f'Size Original:  {size_original} bytes')
print(f'Size Optimized: {size_opt} bytes')
print(f'Optimized predictor achieved a {round((1 - (size_opt/size_original)) * 100, 1)}% reduction in disk usage.')

Size Original:  18612242 bytes
Size Optimized: 574983 bytes
Optimized predictor achieved a 96.9% reduction in disk usage.
